In [196]:
import pandas as pd
import os

In [197]:
headers = ['frame_id', 'track_id', 'X', 'Y', 'W', 'H', 'CONF', 'UNK1', 'UNK2', 'UNK3']
frames = []

for file in os.listdir('./mc/'):
    if not '.txt' in file: continue
    frame = pd.read_csv('./mc/' + file, names = headers)
    frame = frame.filter(regex='^(?!UNK)')
    frame["CAM_ID"] = file.split('.')[0]
    frame["ID"] = [f"{x}_{y}" for x, y in zip(frame["track_id"], frame["frame_id"])]
    
    frames.append(frame)


In [198]:
metaframe = pd.concat(frames)
all_cars = list(metaframe["track_id"].unique())

In [199]:
rows = []

In [200]:
def num_cucus(cucu):
    return sum(["<UNK>" != x[0] for x in cucu])

In [201]:
for car in all_cars:
    filtered = metaframe[metaframe["track_id"] == car]
    track_ids = list(filtered["frame_id"].unique())
    print(len(track_ids))
    for frame in track_ids:
        u = filtered[filtered["frame_id"] == frame]
        CAMS = {f"c{x}gt": ["<UNK>"]*2 for x in [10, 11, 12, 13, 14, 15]}
        for cam in CAMS:
            rowly = u[u['CAM_ID'] == cam].reset_index()
            if len(rowly):
                assert len(rowly) == 1
                CAMS[cam] = [rowly["X"].loc[0] + .5 * rowly["W"].loc[0], rowly["Y"].loc[0] + .5 * rowly["H"].loc[0]]

        newrow = [car, frame,  ]
        for cucu in list(CAMS.values()): 
            #if num_cucus(list(CAMS.values())) > 1:
            newrow += cucu
        rows.append(newrow)
        
    

267
504
275
272
389
404
420
414
215
196
224
466
211
224
63
82
147
219


In [202]:
newheaders = ["track_id", "frame_id",] 

for cam in CAMS: newheaders += [f"X_{cam}", f"Y_{cam}",]
numheaders = []
for cam in CAMS: numheaders += [f"X_{cam}", f"Y_{cam}",]


In [203]:
bigframe = pd.DataFrame(rows, columns = newheaders)

In [204]:
bigframe[bigframe["track_id"] == 258]

,track_id,frame_id,X_c10gt,Y_c10gt,X_c11gt,Y_c11gt,X_c12gt,Y_c12gt,X_c13gt,Y_c13gt,X_c14gt,Y_c14gt,X_c15gt,Y_c15gt
4773,258,1892,<UNK>,<UNK>,<UNK>,<UNK>,1649.0,480.5,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>
4774,258,1893,<UNK>,<UNK>,<UNK>,<UNK>,1651.0,482.5,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>
4775,258,1894,<UNK>,<UNK>,<UNK>,<UNK>,1653.5,485.0,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>
4776,258,1895,<UNK>,<UNK>,<UNK>,<UNK>,1655.5,486.0,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>
4777,258,1896,<UNK>,<UNK>,<UNK>,<UNK>,1655.0,486.5,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4987,258,2020,<UNK>,<UNK>,1833.5,1039.5,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>
4988,258,2021,<UNK>,<UNK>,1814.5,1036.5,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>
4989,258,2022,<UNK>,<UNK>,1814.5,1032.5,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>
4990,258,2023,<UNK>,<UNK>,1815.0,1031.5,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>,<UNK>


In [205]:
stats = {}
for H in numheaders:
    
    indexed = bigframe[H] != '<UNK>'
    stats[H] = {
    "mean": bigframe[indexed][H].mean(),
    "std": bigframe[indexed][H].std()
    }
    
    bigframe[H][indexed] = (bigframe[H][indexed] - stats[H]['mean']) /  stats[H]['std']    


/tmp/ipykernel_40571/2316059807.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bigframe[H][indexed] = (bigframe[H][indexed] - stats[H]['mean']) /  stats[H]['std']


In [206]:
bigframe.to_csv('GT.csv', index = False)
import json
open('normalize.json', 'w').write(json.dumps(stats))

794

In [214]:
for n in range(len(bigframe)):
    if 10 == (sum([x == '<UNK>' for x in (list(bigframe.loc[n]))])):
        bigframe = bigframe.drop(n)
    

In [216]:
bigframe.to_csv('GT.csv', index = False)